### Import Packages

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##### Function to convert to float

In [43]:
def convert_to_float(value):
    try:
        return float(value)
    except ValueError:
        return value  # If conversion fails, return the original value


##### Node class
This class defines a single node. It can be root node, leaf etc depends on the position in the tree

In [44]:
class Node():
    def __init__(self, feature_name=None, threshold=None, left_tree=None, right_tree=None, information_gain=None, leaf_value=None):
        
        self.feature_name = feature_name
        self.threshold = threshold

        self.left = left_tree       # Node
        self.right = right_tree     # Node
        
        self.information_gain = information_gain
        
        #If the node is a leaf node this value will be set
        self.leaf_value = leaf_value

##### Enum SplitCriterion
 Enum is used to define the split criterion ie. gini or entropy

In [45]:
from enum import Enum

class SplitCriterion(Enum):
    GINI = "gini"
    ENTROPY = "entropy"

##### DecisionTreeData class
This class holds all the data for a execution
All the configurations or modification of the data for the classifier is in this class.

In [46]:
class DecisionTreeData:
    def __init__(self, source_data_frame,criterion:SplitCriterion,is_auto_column_type = False,remove_nulls = False) -> None:
        
        if(remove_nulls == True):
            source_data_frame.replace('?', pd.NA, inplace=True)
            source_data_frame.dropna(inplace=True)
            source_data_frame.reset_index(drop=True, inplace=True)
        
        self.source_data_frame = source_data_frame
        self.features_columns_names = []
        self.class_column_name = ''
        self.real_columns_names = []
        self.nominal_column_names = []
        self.tree = Node()
        self.class_values = []
        self.feature_column_values = []
        self.criterion = criterion
        self.training_data = []
        self.test_data = []
        self.is_auto_column_type = is_auto_column_type
        self.folds = {}

    def set_column_type(self,real_columns_names,nominal_column_names):
            self.real_columns_names = real_columns_names
            self.nominal_column_names = nominal_column_names
            
            #Apply the column data type to the data
            self.source_data_frame[real_columns_names] = self.source_data_frame[real_columns_names].apply(lambda x: pd.to_numeric(x, errors='coerce'))

    def process(self):
        self.feature_column_values = self.source_data_frame.iloc[:, :-1]
        self.class_values = self.source_data_frame.iloc[:, -1]

        self.class_column_name = self.source_data_frame.columns[-1]

        self.features_columns_names = self.feature_column_values.columns
        
        # If is_auto_column_type is true then system will calculate the column data type
        # Otherwise we need to specify the colum data type
        if(self.is_auto_column_type == True):
            self.identify_colum_types()
            
    def convert_nominal_values_to_real(self):
        self.source_data_frame = pd.get_dummies(self.source_data_frame, columns=self.nominal_column_names)
        
    def identify_colum_types(self):
        column_types = self.feature_column_values.dtypes

        self.real_columns_names = [col for col, dtype in column_types.items() if dtype in ['int64', 'float64']]
        self.nominal_column_names = [col for col, dtype in column_types.items() if dtype == 'object']
        
    def train_test_split(self):
        shuffled_data = self.source_data_frame.sample(frac=1)
        split_point = int(0.8 * len(self.source_data_frame))

        self.training_data = shuffled_data.iloc[:split_point]
        self.test_data = shuffled_data.iloc[split_point:]
        
    def generate_folds(self,fold_count):
        rows_per_fold = len(self.source_data_frame)/fold_count
        self.folds = {}
        shuffled_data = self.source_data_frame.sample(frac=1)
        
        for i in range(fold_count):
            start_index = int(i * rows_per_fold)
            end_index = int((i + 1) * rows_per_fold)
            subset = shuffled_data.iloc[start_index:end_index]
            self.folds[i] = subset
            
    def reset_training_test_data(self,training_data,test_data):
        self.training_data = training_data
        self.test_data = test_data
        

##### DecisionTreeSplit
This class is a model class for giving a format to the each split

In [47]:
class DecisionTreeSplit:
    def __init__(self, feature_name = None,threshold = None,left_dataset = None,right_dataset = None,information_gain = -float("inf")):
        self.feature_name = feature_name
        self.threshold = threshold
        self.left_dataset = left_dataset
        self.right_dataset = right_dataset
        self.information_gain = information_gain


##### DecisionTreeClassifier Class
This is the decision tree classifier class works with the data - DecisionTreeData

In [48]:
class DecisionTreeClassifier:
    def __init__(self,decisionTreeData:DecisionTreeData,min_samples = 2,max_depth = 2) -> None:
        self.decisionTreeData = decisionTreeData
        self.min_samples = min_samples
        self.max_depth =  max_depth
        
    def execute(self):
        self.decisionTreeData.tree = self.build_tree(self.decisionTreeData.training_data,0)
        
    def calculate_entropy(self,labels):
        _ , counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy
    
    def calculate_gini(self,labels):
        _ , counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        gini_impurity = 1 - np.sum(probabilities**2)
        return gini_impurity
    
    def split_real_valued_feature(self, dataset, feature_index):
        mean_value = dataset[feature_index].mean()
        dataset_left = dataset[dataset[feature_index] < mean_value]
        dataset_right = dataset[dataset[feature_index] >= mean_value]
        return dataset_left, dataset_right, mean_value

    def split_nominal_valued_feature(self, dataset, feature_index, threshold):
        dataset_left = dataset[dataset[feature_index] == threshold]
        dataset_right = dataset[dataset[feature_index] != threshold]
        return dataset_left, dataset_right
    
    def get_majority_class(self, data):
        return data[self.decisionTreeData.class_column_name].mode().iloc[0]
    
    def build_tree(self, data, depth=0):
        decision_tree_split = self.find_best_split(data)

        if depth >= self.max_depth or len(data) <= self.min_samples:
            majority_class = self.get_majority_class(data)
            return Node(leaf_value=majority_class)

        if(decision_tree_split.information_gain > 0):
            left_tree = self.build_tree(decision_tree_split.left_dataset, depth + 1)
            right_tree = self.build_tree(decision_tree_split.right_dataset, depth + 1)

            return Node(decision_tree_split.feature_name,
                        decision_tree_split.threshold,
                        left_tree,
                        right_tree,
                        decision_tree_split.information_gain)
        
        return Node(leaf_value=self.get_majority_class(data))
        
    def information_gain(self, parent, left_child, right_child):        
        weight_l = len(left_child) / len(parent)
        weight_r = len(right_child) / len(parent)
        if self.decisionTreeData.criterion == SplitCriterion.GINI:
            gain = self.calculate_gini(parent) - (weight_l*self.calculate_gini(left_child) + weight_r*self.calculate_gini(right_child))
            
        else:
            gain = self.calculate_entropy(parent) - (weight_l*self.calculate_entropy(left_child) + weight_r*self.calculate_entropy(right_child))
            
        return gain

    def find_best_split(self,dataset):
        best_feature_split = DecisionTreeSplit()
        
        for feature_name in self.decisionTreeData.features_columns_names:
            feature_values = dataset[feature_name]
            
            if(self.decisionTreeData.real_columns_names.__contains__(feature_name)):
                left_dataset,right_dataset,threshold =  self.split_real_valued_feature(dataset,feature_name)
                self.calculate_best_feature(dataset, best_feature_split, feature_name, left_dataset, right_dataset, threshold)
               
            else:
                for threshold in np.unique(feature_values):
                    left_dataset, right_dataset = self.split_nominal_valued_feature(dataset, feature_name, threshold)
                    self.calculate_best_feature(dataset, best_feature_split, feature_name, left_dataset, right_dataset, threshold)
                        
        return best_feature_split

    def calculate_best_feature(self, dataset, best_split, feature_name, left_dataset, right_dataset, threshold):
        parent_data_class_values= dataset[self.decisionTreeData.class_column_name]
        left_class_values  = left_dataset[self.decisionTreeData.class_column_name]
        right_class_values = right_dataset[self.decisionTreeData.class_column_name]
        
        current_info_gain = self.information_gain(parent_data_class_values, left_class_values, right_class_values) 
                    
        if current_info_gain > best_split.information_gain:
            best_split.feature_name = feature_name
            best_split.threshold = threshold
            best_split.left_dataset = left_dataset.copy()
            best_split.right_dataset = right_dataset.copy()
            best_split.information_gain = current_info_gain
      
    

##### DecisionTreePredictor class
This class takes the generated tree as input and predict the output class based on the tree.
It also calculates the accuracy

In [49]:
class DecisionTreePredictor:
    def __init__(self, tree):
        self.tree = tree

    def predict_single(self, features, tree):
        if tree.leaf_value is not None:
            return tree.leaf_value
        else:
            if features[tree.feature_name] < tree.threshold:
                return self.predict_single(features, tree.left)
            else:
                return self.predict_single(features, tree.right)

    def predict(self, data):
        predictions = []

        for _, row in data.iterrows():
            prediction = self.predict_single(row, self.tree)
            predictions.append(prediction)

        result_data_frame = data.copy()
        result_data_frame['predicted_value'] = predictions
        return result_data_frame
    
    def calculate_accuracy(self,predictions):
        predicted_values = predictions.iloc[:, -2]  # Assuming predicted value is the second to last column
        actual_values = predictions.iloc[:, -1]     # Assuming original label is the last column
        
        correct_predictions = sum(predicted_values == actual_values)
        total_instances = len(actual_values)
        accuracy = correct_predictions / total_instances
        return accuracy


##### SkLearnClassifier Class
Scikit Learn Classifier Implementation.
This class handles all the changes need to implement the scikit learn classifier and it takes input as DecisionTreeData

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier as SklearnDecisionTreeClassifier
from sklearn.metrics import accuracy_score

class SkLearnClassifier:
    def execute(self, data:DecisionTreeData,max_depth):
        label_encoder = LabelEncoder()

        combined_data = pd.concat([data.training_data, data.test_data], axis=0)

        combined_data[data.class_column_name] = label_encoder.fit_transform(combined_data[data.class_column_name])

        # Encode nominal columns
        for column_name in data.nominal_column_names:
            combined_data[column_name] = label_encoder.fit_transform(combined_data[column_name])

        # Split combined data back into training and test sets
        training_data = combined_data[:len(data.training_data)]
        test_data = combined_data[len(data.training_data):]

        training_data_features = training_data.drop(columns=[data.class_column_name])
        training_data_class = training_data[data.class_column_name]

        test_data_features = test_data.drop(columns=[data.class_column_name])
        test_data_class = test_data[data.class_column_name]

        classifier = SklearnDecisionTreeClassifier(criterion = data.criterion.value,max_depth=max_depth)
        classifier.fit(training_data_features, training_data_class)

        sklearn_predictions = classifier.predict(test_data_features)

        return accuracy_score(test_data_class, sklearn_predictions)


##### Common function to train, predict based on data and perform cross validation for both classifiers


In [51]:
def decision_tree_predict_accuracy(max_depth, min_samples, data:DecisionTreeData):
    classifier = DecisionTreeClassifier(data, min_samples, max_depth)
    classifier.execute()

    predictor = DecisionTreePredictor(data.tree)
    predictions =  predictor.predict(data.test_data)
    accuracy = predictor.calculate_accuracy(predictions)

    #Scikit Learn Classifier
    sklearn_accuracy = SkLearnClassifier().execute(data,max_depth)
    return accuracy,sklearn_accuracy

def calculate_data_set(data:DecisionTreeData, index,fold_count):
    test_data = data.folds[index]
    other_keys = [key for key in range(fold_count) if key != index]
    training_data = pd.concat([data.folds[key] for key in other_keys], ignore_index=True)
    return test_data,training_data

def perform_cross_validation(max_depth, min_samples, data:DecisionTreeData,fold_count):
    data.generate_folds(fold_count)

    accuracy_array = []
    sklearn_accuracy_array = []
    for index in range(fold_count):
        test_data, training_data = calculate_data_set(data, index,fold_count)
        data.reset_training_test_data(training_data,test_data)
        accuracy, sklearn_accuracy = decision_tree_predict_accuracy(max_depth,min_samples, data)
        accuracy_array.append(accuracy)
        sklearn_accuracy_array.append(sklearn_accuracy)
        print(f"Accuracy: {accuracy * 100:.4f}%, Scikit Learn's Accuracy: {sklearn_accuracy * 100:.4f}%")

    print(f" \nAverage Decision Tree Accuracy: {np.average(accuracy_array) * 100:.4f}%")
    print(f"Average Scikit Learn's Decision Tree Accuracy: {np.average(sklearn_accuracy_array) * 100:.4f}%")


# Decision Tree Classification Predictor
`Configure the column names based on the data type into two arrays`
1. real_valued_columns
2. nominal_valued_columns

Configure the max_depth and min_samples for the tree.


### 1. Iris Data

In [52]:
real_valued_columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

source_data_frame = pd.read_csv(url, names=column_names)

nominal_valued_columns = []
max_depth = 5
min_samples = 3

# Config the data based on need
data = DecisionTreeData(source_data_frame,SplitCriterion.ENTROPY,is_auto_column_type=False)
data.set_column_type(real_valued_columns,nominal_valued_columns)
data.process()
data.train_test_split()

accuracy, sklearn_accuracy = decision_tree_predict_accuracy(max_depth,min_samples, data)

print(f"Decision Tree Accuracy: {accuracy * 100:.4f}%")
print(f"Scikit Learn's Decision Tree accuracy: {sklearn_accuracy * 100:.4f}% \n")

perform_cross_validation(max_depth,min_samples, data,fold_count=5)


Decision Tree Accuracy: 90.0000%
Scikit Learn's Decision Tree accuracy: 93.3333% 

Accuracy: 90.0000%, Scikit Learn's Accuracy: 96.6667%
Accuracy: 93.3333%, Scikit Learn's Accuracy: 93.3333%
Accuracy: 93.3333%, Scikit Learn's Accuracy: 93.3333%
Accuracy: 93.3333%, Scikit Learn's Accuracy: 100.0000%
Accuracy: 96.6667%, Scikit Learn's Accuracy: 93.3333%
 
Average Decision Tree Accuracy: 93.3333%
Average Scikit Learn's Decision Tree Accuracy: 95.3333%
